In [119]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%matplotlib inline
import cv2, keras
#os.listdir()

In [11]:
label = pd.read_csv('Label.csv',index_col=False)

In [22]:
Breed = label['breed'].unique()
le = LabelEncoder()
le.fit(Breed)
label['breed'] = le.transform(label['breed'])

In [23]:
label.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,19
1,001513dfcb2ffafc82cccf4d8bbaba97,37
2,001cdf01b096e06d78e9e5112d419397,85
3,00214f311d5d2247d5dfe4fe24b2303d,15
4,0021f9ceb3235effd7fcde7f7538ed62,49


In [52]:
label[label['id']=='000bec180eb18c7604dcecc8fe0dba07'].iloc[0,1]


19

In [78]:
os.chdir('C:\\Users\\sudramak\\Anaconda-DS\\SuperDS projects\\Kaggle Competition\\Dog breed Identification\\\\Train\\train')

In [79]:
#load the images along with X(train) and Y(target)
import glob
X = []
Y = []
for filename in glob.glob('*.jpg'):
    n = cv2.imread(filename,-1)
    if n is not None:
         X.append(cv2.resize(n,(256,256)))
         tempfil = filename[:len(filename)-4]
         Y.append(label[label['id']==tempfil].iloc[0,1])
#img.append(cv2.imread('000bec180eb18c7604dcecc8fe0dba07.jpg',0))
#print(img)

In [139]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [140]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [141]:
y_train = keras.utils.to_categorical(y_train, 120)
y_test = keras.utils.to_categorical(y_test, 120)

In [142]:
x_train.shape

(8177, 256, 256, 3)

In [143]:
y_train.shape

(8177, 120)

In [49]:
import matplotlib.image as mpimg
img = []
for filename in glob.glob('*.jpg'):
    n = mpimg.imread(filename,0)
    if n is not None:
         img.append(n)
         #print(filename)

In [101]:
from keras.models import Model,Sequential
from keras.layers import Convolution2D,Conv2D,MaxPooling2D,Flatten,Dense,Input,Conv2DTranspose,concatenate,Activation
from keras.layers import BatchNormalization
from keras.optimizers import Adam

In [145]:
img_size = 256
row_img = img_size
col_img = img_size
batch_size = 50
nb_epoch = 10
lr = 0.001
img = Input(shape=(row_img,col_img,3))

conv1 = Conv2D(32,(3,3),padding='valid',strides=(2,2))(img)
conv1 = BatchNormalization()(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

conv2 = Conv2D(64,(3,3),padding='valid',strides=(2,2))(pool1)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

conv3 = Conv2D(128,(3,3),padding='valid',strides=(2,2))(pool2)
conv3 = BatchNormalization()(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
                                      
Flat3 = Flatten()(pool3)
Dense3 = Dense(10000)(Flat3)
Activ3 = Activation('relu')(Dense3)
Dense4 = Dense(120)(Activ3)
Activ4 = Activation('softmax')(Dense4)                                      

In [146]:
model = Model(inputs=img, outputs=Activ4)
model.compile(optimizer=Adam(lr=lr),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          verbose=1,
          validation_data=(x_test, y_test))


C:\Users\sudramak\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 579s 71ms/step - loss: 6.4115 - acc: 0.0139 - val_loss: 4.7540 - val_acc: 0.0200
Epoch 2/10
8177/8177 [==============================] - 556s 68ms/step - loss: 4.6532 - acc: 0.0229 - val_loss: 4.8389 - val_acc: 0.0161
Epoch 3/10
8177/8177 [==============================] - 579s 71ms/step - loss: 4.4282 - acc: 0.0410 - val_loss: 4.4679 - val_acc: 0.0391
Epoch 4/10
8177/8177 [==============================] - 605s 74ms/step - loss: 4.2267 - acc: 0.0572 - val_loss: 4.4430 - val_acc: 0.0391
Epoch 5/10
8177/8177 [==============================] - 579s 71ms/step - loss: 4.0232 - acc: 0.0857 - val_loss: 4.5316 - val_acc: 0.0367
Epoch 6/10
8177/8177 [==============================] - 633s 77ms/step - loss: 3.8085 - acc: 0.1189 - val_loss: 4.5664 - val_acc: 0.0421
Epoch 7/10
8177/8177 [==============================] - 619s 76ms/step - loss: 3.5736 - acc: 0.1508 - val_loss: 4.3328 - val_acc: 